In [1]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\jianbai\Desktop\protfolio\data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head()


,Company,Date,Close/Last,Volume,Open,High,Low
0,AAPL,07/17/2023,$193.99,50520160,$191.90,$194.32,$191.81
1,AAPL,07/14/2023,$190.69,41616240,$190.23,$191.1799,$189.63
2,AAPL,07/13/2023,$190.54,41342340,$190.50,$191.19,$189.78
3,AAPL,07-12-2023,$189.77,60750250,$189.68,$191.70,$188.47
4,AAPL,07-11-2023,$188.08,46638120,$189.16,$189.30,$186.60


In [2]:
# Clean and prepare the data
# Convert 'Close/Last' to numeric values after removing the '$' sign
data['Close/Last'] = data['Close/Last'].replace('[\$,]', '', regex=True).astype(float)

# Convert 'Date' to a datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort data by 'Company' and 'Date'
data.sort_values(by=['Company', 'Date'], inplace=True)

# Calculate daily returns
data['Daily Return'] = data.groupby('Company')['Close/Last'].pct_change()

# Display the cleaned data with daily returns
data.head()


,Company,Date,Close/Last,Volume,Open,High,Low,Daily Return
2515,AAPL,2013-07-18,15.4199,218632537,$15.4779,$15.5311,$15.3789,NaN
2514,AAPL,2013-07-19,15.1768,268548901,$15.4679,$15.4993,$15.1554,-0.015765
2513,AAPL,2013-07-22,15.2254,207648981,$15.3379,$15.3482,$15.1953,0.003202
2512,AAPL,2013-07-23,14.9639,354477618,$15.2143,$15.2486,$14.9539,-0.017175
2511,AAPL,2013-07-24,15.7325,591624923,$15.6761,$15.8782,$15.545,0.051364


In [7]:
# Calculate the volatility (standard deviation) of each stock
volatility = data.groupby('Company')['Daily Return'].std()

# Calculate the correlation matrix of the daily returns
correlation_matrix = data.pivot(index='Date', columns='Company', values='Daily Return').corr()

# Display the volatility and the correlation matrix
volatility, correlation_matrix


(Company
 AAPL    0.022074
 AMD     0.046147
 AMZN    0.025579
 CSCO    0.019148
 META    0.030803
 MSFT    0.020156
 NFLX    0.035189
 QCOM    0.027961
 SBUX    0.020388
 TSLA    0.045155
 Name: Daily Return, dtype: float64,
 Company      AAPL       AMD      AMZN      CSCO      META      MSFT      NFLX  \
 Company                                                                         
 AAPL     1.000000  0.366190  0.502091  0.471512  0.462638  0.628389  0.364137   
 AMD      0.366190  1.000000  0.378289  0.332707  0.329445  0.411337  0.324672   
 AMZN     0.502091  0.378289  1.000000  0.336830  0.500733  0.581794  0.510616   
 CSCO     0.471512  0.332707  0.336830  1.000000  0.290583  0.511794  0.271849   
 META     0.462638  0.329445  0.500733  0.290583  1.000000  0.503504  0.432211   
 MSFT     0.628389  0.411337  0.581794  0.511794  0.503504  1.000000  0.435089   
 NFLX     0.364137  0.324672  0.510616  0.271849  0.432211  0.435089  1.000000   
 QCOM     0.506642  0.419159  0.3716

In [8]:
from scipy.optimize import LinearConstraint, minimize
import numpy as np

# Number of assets
n_assets = len(volatility)

# Convert volatility from Series to numpy array
volatility_array = volatility.values

# Convert correlation matrix to numpy array
correlation_matrix_array = correlation_matrix.values

# Covariance matrix calculation
covariance_matrix = np.outer(volatility_array, volatility_array) * correlation_matrix_array

# Risk contribution function for each asset
def risk_contribution(weights):
    total_portfolio_var = np.dot(weights.T, np.dot(covariance_matrix, weights))
    marginal_contrib = np.dot(covariance_matrix, weights)
    risk_contrib = np.multiply(marginal_contrib, weights) / total_portfolio_var
    return risk_contrib

# Objective function: minimize the sum of squared differences between risk contributions
def objective(weights):
    risk_contrib = risk_contribution(weights)
    return np.sum((risk_contrib - 1 / n_assets) ** 2)

# Constraints: Weights sum to 1 and are non-negative
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights is 1
    {'type': 'ineq', 'fun': lambda x: x}             # Weights are non-negative
]

# Initial guess (equal weights)
initial_weights = np.ones(n_assets) / n_assets

# Minimize the objective function using a method that can handle constraints
opt_result = minimize(objective, initial_weights, method='SLSQP', constraints=constraints)

# Optimized weights
optimized_weights = opt_result.x

# Display the optimized weights
optimized_weights, opt_result.success, opt_result.message


(array([0.10943112, 0.06205831, 0.09683858, 0.15046974, 0.08701927,
        0.11384166, 0.08006371, 0.09803683, 0.13570853, 0.06653225]),
 True,
 'Optimization terminated successfully')

In [11]:
# Calculate the actual risk contributions of each asset in the optimized portfolio
actual_risk_contributions = risk_contribution(optimized_weights)

# Ideal average risk contribution (1/N)
ideal_risk_contribution = 1 / n_assets

# Comparison of actual risk contributions with the ideal contribution
risk_contribution_comparison = pd.DataFrame({
    'Asset': volatility.index,
    'Optimized Weight (%)': optimized_weights * 100,
    'Actual Risk Contribution (%)': actual_risk_contributions * 100,
    'Ideal Risk Contribution (%)': ideal_risk_contribution * 100
})

# Calculate total portfolio risk (volatility) for the optimized portfolio
total_portfolio_volatility = np.sqrt(np.dot(optimized_weights.T, np.dot(covariance_matrix, optimized_weights)))

risk_contribution_comparison, total_portfolio_volatility


(  Asset  Optimized Weight (%)  Actual Risk Contribution (%)  \
 0  AAPL             10.943112                      9.991041   
 1   AMD              6.205831                      9.979666   
 2  AMZN              9.683858                     10.016868   
 3  CSCO             15.046974                     10.008809   
 4  META              8.701927                      9.987588   
 5  MSFT             11.384166                      9.984720   
 6  NFLX              8.006371                     10.003142   
 7  QCOM              9.803683                     10.026176   
 8  SBUX             13.570853                     10.017363   
 9  TSLA              6.653225                      9.984627   
 
    Ideal Risk Contribution (%)  
 0                         10.0  
 1                         10.0  
 2                         10.0  
 3                         10.0  
 4                         10.0  
 5                         10.0  
 6                         10.0  
 7                    